In [1]:
import os

from keras.callbacks import ModelCheckpoint

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import glob
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from numpy import genfromtxt
from keras.utils import np_utils
import tensorflow
import numpy
import os
import cv2
import sys
from keras import backend as K
K.set_image_dim_ordering('th')

cwd = os.getcwd()

Using TensorFlow backend.


In [2]:
#Part 1 -> defines
img_rows, img_cols = 128, 128
nb_filters = 32 # nr of conv filters to use
nb_pool = 2 # size of pooling area
nb_conv = 3 # convolution kernel size

In [3]:
#Part 2 -> get data
def get_im(path):
    # Load as grayscale
    print("Path: ", path)
    img = cv2.imread(path, 0)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

def getDataManually(path):
    X = []
    y = []
    
    for dir in os.listdir(path):
        crtPath = os.path.join(path, dir, "*.png")
        
        files = glob.glob(crtPath)
        for fl in files:
             # X
             fl = os.path.join(cwd, fl)
             img = get_im(fl)
             X.append(img)
            
             # Y
             y.append(int(dir))
             print("Picture added from path: ", fl)

    return X, y

def getData():
    train_datagen = ImageDataGenerator()
    test_datagen = ImageDataGenerator()

    x_train, y_train = getDataManually(os.path.join("dataset", "training_set"))
    x_test, y_test = getDataManually(os.path.join("dataset", "test_set"))
    
    x_train = numpy.array(x_train)
    x_train = numpy.reshape(x_train, (len(x_train), 1, img_rows, img_cols))
    x_train = x_train.astype("float64")
    train_datagen.fit(x_train, augment = False)

    x_test = numpy.array(x_test)
    x_test = numpy.reshape(x_test, (len(x_test), 1, img_rows, img_cols))
    x_test = x_test.astype("float64")
    test_datagen.fit(x_test, augment = False)

    return x_train, y_train, x_test, y_test, train_datagen, test_datagen

In [4]:
#Part 3 -> get the model
def getModel():
    classifier = Sequential()
    classifier.add(
    Convolution2D(32, 3, 3, input_shape=(1, 128, 128), activation='relu')
    )
    classifier.add(Dropout(0.2))
    classifier.add(Convolution2D(64, 3, 3, activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Convolution2D(64, 3, 3, activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Convolution2D(128, 3, 3, activation='relu'))

    classifier.add(Flatten())
    classifier.add(Dense(2048, activation='relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(1024, activation='relu'))
    classifier.add(Dense(2048, activation='relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(output_dim = 999, activation = 'softmax'))

    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
#Part 4 -> fit the model
def fitModel(x_train, y_train, x_test, y_test, train_datagen, test_datagen, classifier):
    filepath = sys.argv[0] + "-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    classifier.fit_generator(train_datagen.flow(x_train, y_train, batch_size= 32),
                         samples_per_epoch = 10000,
                         nb_epoch = 80,
                         validation_data = test_datagen.flow(x_test, y_test),
                         nb_val_samples = 5000,
                         callbacks = callbacks_list
                        )

In [ ]:
#Part 5 -> run everything
x_train, y_train, x_test, y_test, train_datagen, test_datagen = getData()

#Encode y_train, y_test
y_train = np_utils.to_categorical(y_train, nb_classes = 999)
y_test = np_utils.to_categorical(y_test, nb_classes = 999)

classifier = getModel()
fitModel(x_train, y_train, x_test, y_test, train_datagen, test_datagen, classifier)

('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/95554.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/95554.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/35189.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/35189.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/27500.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/27500.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/80432.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/80432.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/17963.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/17963.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/862/78232.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/862/78232.png'

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/49515.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/17485.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/17485.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/22281.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/22281.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/84087.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/84087.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/86340.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/86340.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/97234.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/125/97234.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/125/453.png')


('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/50805.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/57602.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/57602.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/60821.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/60821.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/52882.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/52882.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/69272.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/69272.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/85257.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/893/85257.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/893/76986.png'

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/82956.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/61484.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/61484.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/83825.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/83825.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/13389.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/13389.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/9352.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/9352.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/23585.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/270/23585.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/270/34033.png')


('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/7448.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/7448.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/25367.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/25367.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/44663.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/44663.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/74748.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/74748.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/16087.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/16087.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/131/30467.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/131/30467.png')


...

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/95278.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/23311.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/23311.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/17617.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/17617.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/86658.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/86658.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/461.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/461.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/86949.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/817/86949.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/817/25846.png')
('

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/10413.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/18001.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/18001.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/14258.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/14258.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/49378.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/49378.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/90808.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/90808.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/53488.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/939/53488.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/939/80901.png'

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/50220.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/36222.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/36222.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/57491.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/57491.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/23863.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/23863.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/20405.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/20405.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/75397.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/599/75397.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/599/77856.png'

('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/9396.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/7476.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/7476.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/98305.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/98305.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/64186.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/64186.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/5217.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/5217.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/57358.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/818/57358.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/818/8520.png')
('Pi

('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/72338.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/72338.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/52935.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/52935.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/56446.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/56446.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/28288.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/28288.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/89158.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/89158.png')
('Path: ', '/docker_shared/12Captcha/dataset/training_set/297/67481.png')
('Picture added from path: ', '/docker_shared/12Captcha/dataset/training_set/297/67481.png'